In [7]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout,GRU
from keras import optimizers 
from sklearn.metrics import mean_squared_error

seed = 1234
np.random.seed(seed)
plt.style.use('ggplot')

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [8]:
from predata import Data
data = Data('windspeed.csv', column_name='WindSpeed')


In [9]:
x_train = data.x_train
y_train = data.y_train

x_test = data.x_test
y_test = data.y_test

x_val = data.x_val
y_val = data.y_val

In [10]:
# Hyperparameters
learning_rate = 0.0001
hidden_unit = 64
batch_size= 32
epoch = 50

# Architecture Gated Recurrent Unit
regressorGRU = Sequential()

# First GRU layer with dropout
regressorGRU.add(GRU(units=hidden_unit, return_sequences=True, input_shape=(x_train.shape[1],1), activation = 'relu'))
regressorGRU.add(Dropout(0.2))
# Second GRU layer with dropout
regressorGRU.add(GRU(units=hidden_unit, return_sequences=True, activation = 'relu'))
regressorGRU.add(Dropout(0.2))
# Third GRU layer with dropout
regressorGRU.add(GRU(units=hidden_unit, return_sequences=False, activation = 'relu'))
regressorGRU.add(Dropout(0.2))

# Output layer
regressorGRU.add(Dense(units=1))

# Compiling the Gated Recurrent Unit
regressorGRU.compile(optimizer=tf.keras.optimizers.Adam(lr=learning_rate),loss='mean_squared_error')

# Fitting ke data training dan data validation
pred = regressorGRU.fit(x_train, y_train, validation_data=(x_val,y_val), batch_size=batch_size, epochs=epoch)

Epoch 1/50
1229/1229 [==============================] - 19s 9ms/step - loss: 0.0079 - val_loss: 7.7451e-04
Epoch 2/50
1229/1229 [==============================] - 12s 10ms/step - loss: 0.0027 - val_loss: 6.4111e-04
Epoch 3/50
1229/1229 [==============================] - 11s 9ms/step - loss: 0.0022 - val_loss: 6.8910e-04
Epoch 4/50
1229/1229 [==============================] - 11s 9ms/step - loss: 0.0021 - val_loss: 6.8488e-04
Epoch 5/50
1229/1229 [==============================] - 11s 9ms/step - loss: 0.0020 - val_loss: 5.7932e-04
Epoch 6/50
1229/1229 [==============================] - 11s 9ms/step - loss: 0.0020 - val_loss: 7.2261e-04
Epoch 7/50
1229/1229 [==============================] - 12s 10ms/step - loss: 0.0019 - val_loss: 6.7420e-04
Epoch 8/50
1229/1229 [==============================] - 13s 10ms/step - loss: 0.0020 - val_loss: 6.8401e-04
Epoch 9/50
1229/1229 [==============================] - 12s 10ms/step - loss: 0.0020 - val_loss: 5.9476e-04
Epoch 10/50
1229/1229 [==========

In [11]:
regressorGRU.save('windspeed.h5')

In [13]:
y_pred_test = regressorGRU.predict(x_test)

246/246 [==============================] - 1s 3ms/step


In [14]:
# Invert normalization min-max
y_pred_invert_norm = data.scaler.inverse_transform(y_pred_test)

In [17]:
def MAPE(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))*100
    return mape
MAPE(y_test, y_pred_invert_norm)

16103.691464022071

In [18]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(
    y_true=y_test,
    y_pred=y_pred_invert_norm
)

0.7062990806259337